# Projet : Statistique en Grande Dimension - Exercice 1

Solène Lesage

### Question 2

#### Partie 2 : 

Dans cette partie, on souhaite tester différentes paramètres concernant comment régulariser en utilisant l'algorithme XGBoost.

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

from xgboost import XGBRegressor
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")

### Ilustration sur l'hyperparamètre : max_depth

In [18]:
x_train = pd.DataFrame({"A" : [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 9.0]})
y_train = pd.DataFrame({"Y" : [2.0, 4.0, 6.0, 8.0, 20.0, 24.0, 28.0, 36.0]})

On va faire varier max_depth pour voir l'impact de ce paramètre, les autres paramètres sont fixés à 0.
C'est juste un exemple illustratif.

In [19]:
# Choix de profondeur maximale à 3
model = XGBRegressor(n_estimators=1,
                     learning_rate=1.,
                     base_score=0,
                     max_depth=3,
                     gamma=0,
                     reg_alpha=0,
                     reg_lambda=0)

model.fit(x_train, y_train['Y'])
pred = model.predict(x_train)
print(pred) 
# -> [ 2.  4.  6.  8. 20. 24. 28. 36.]

[ 2.  4.  6.  8. 22. 22. 28. 36.]


Ce modèle ne permet pas un apprentissage complet et ne fournit pas une prédiction exacte comparé à l'échantillon y_train.

In [20]:
# Choix de profondeur maximale à 4
model1 = XGBRegressor(n_estimators=1,
                     learning_rate=1.,
                     base_score=0,
                     max_depth=4,
                     gamma=0,
                     reg_alpha=0,
                     reg_lambda=0)

model1.fit(x_train, y_train['Y'])
pred1 = model1.predict(x_train)
print(pred1) 
# -> [ 2.  4.  6.  8. 20. 24. 28. 36.]

[ 2.  4.  6.  8. 20. 24. 28. 36.]


Le second modèle avec un choix de profondeur maximale à 4, qui peut donc prédire jusqu'à 16 valeurs différentes (2^4) donne les prédictions souhaitées.

### Ilustration sur l'hyperparamètre : learning rate

In [24]:
x_train = pd.DataFrame({"A" : [3.0, 2.0, 1.0, 4.0, 5.0, 6.0, 7.0]})
y_train = pd.DataFrame({"Y" : [3.0, 2.0, 1.0, 4.0, 5.0, 6.0, 7.0]})

In [25]:
# Learning rate égal à 1
model = XGBRegressor(n_estimators=1,
                     learning_rate=1.,
                     base_score=0,
                     max_depth=3,
                     gamma=0,
                     reg_alpha=0,
                     reg_lambda=0)

model.fit(x_train, y_train['Y'])
pred = model.predict(x_train)
print(pred) 
# -> [3. 2. 1. 4. 5. 6. 7.]

[3. 2. 1. 4. 5. 6. 7.]


Ce modèle montre que les poids sont pas modifiés.

In [26]:
# Learning rate égal à 0.8
model1 = XGBRegressor(n_estimators=1,
                     learning_rate=0.8,
                     base_score=0,
                     max_depth=3,
                     gamma=0,
                     reg_alpha=0,
                     reg_lambda=0)

model1.fit(x_train, y_train['Y'])
pred1 = model1.predict(x_train)
print(pred1) 
# -> [2.4 1.6 0.8 3.2 4.  4.8 5.6]

[2.4 1.6 0.8 3.2 4.  4.8 5.6]


On voit bien que les poids ont été multipliés par 0.8 (learning rate).
Pour obtenir une meilleure prédiction, on pourrait augmenter le nombre d'arbres.

### Ilustration sur le paramètre de régularisation : gamma

Le code suivant utilise l'algorithme XGBoost pour classer un entier : soit il est positif soit négatif.

In [37]:
x_train = pd.DataFrame({"A" : [-3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0]})
y_train = pd.DataFrame({"Y" : [-1, -1, -1, 1.0, 1.0, 1.0, 1.0]})
y_train['Y'] = y_train['Y'] + np.random.normal(0, .1, y_train.shape[0])

Les données d'entrainement associe -1 aux entiers négatifs et à 1 pour ceux positifs ou nuls.
Le rajout de la loi normale est pour mettre en évidence l'exemple.

In [38]:
# overfitting
model = XGBRegressor(n_estimators=1,
                     learning_rate=1.,
                     base_score=0,
                     max_depth=3,
                     gamma=0,
                     reg_alpha=0,
                     reg_lambda=0)

model.fit(x_train, y_train['Y'])
pred = model.predict(x_train)
print(pred) 
# -> [-0.9603168  -1.1339611  -0.98575723  1.1199436   0.944808    0.944808 1.1765499 ]

[-0.9603168  -1.1339611  -0.98575723  1.1199436   0.944808    0.944808
  1.1765499 ]


On obtient 7 valeurs différentes et autant de feuilles.

In [39]:
# regularisation gamma = 1
model1 = XGBRegressor(n_estimators=1,
                     learning_rate=1.,
                     base_score=0,
                     max_depth=3,
                     gamma=1,
                     reg_alpha=0,
                     reg_lambda=0)

model1.fit(x_train, y_train['Y'])
pred1 = model1.predict(x_train)
print(pred1) 
# -> [-1.0266783 -1.0266783 -1.0266783  1.0465274  1.0465274  1.0465274 1.0465274]

[-1.0266783 -1.0266783 -1.0266783  1.0465274  1.0465274  1.0465274
  1.0465274]


L'ajout d'un niveau à l'arbre se fait que si le gain est supérieure à gamma ici 1.
Les prédictions obtenues montrent seulement deux labels : -1.0266 et 1.0465.
Le modèle a bien capturé le problème dans sa globalité qui a classer en seulement deux catégories les nombres : positifs et négatifs.
On a bien réduit le sur-apprentissage en jouant sur la structure de l'arbre.

### Ilustration sur le paramètre de régularisation : lambda

In [40]:
# sur-apprentissage évident avec lambda = 0
model = XGBRegressor(n_estimators=1,
                     learning_rate=1.,
                     base_score=0,
                     max_depth=3,
                     gamma=0,
                     reg_alpha=0,
                     reg_lambda=0)

model.fit(x_train, y_train['Y'])
pred = model.predict(x_train)
print(pred) 
# -> [-0.9603168  -1.1339611  -0.98575723  1.1199436   0.944808    0.944808 1.1765499 ]

[-0.9603168  -1.1339611  -0.98575723  1.1199436   0.944808    0.944808
  1.1765499 ]


On observe un sur-apprentissage nette car chaque entrée a son propre label : deux labels suffiraient.

In [41]:
# regularisation lambda=1
model1 = XGBRegressor(n_estimators=1,
                     learning_rate=1.,
                     base_score=0,
                     max_depth=3,
                     gamma=0,
                     reg_alpha=0,
                     reg_lambda=1)

model1.fit(x_train, y_train['Y'])
pred1 = model1.predict(x_train)
print(pred1) 
# -> [-0.7700088 -0.7700088 -0.7700088  0.8372219  0.8372219  0.8372219 0.8372219]

[-0.7700088 -0.7700088 -0.7700088  0.8372219  0.8372219  0.8372219
  0.8372219]


Le second modèle permet de limiter le sur-apprentissage et assure la création d'un modèle a 2 feuilles.
Le paramètre lambda a une influence sur la structure du modèle.

### Ilustration sur le paramètre de régularisation : alpha

In [43]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
# Régularisation de type L2 : alpha = 1
model = XGBClassifier(n_estimators=20,
                      learning_rate=0.3,
                      max_depth=3,
                      gamma=0,
                      reg_alpha=0,
                      reg_lambda=2,
                      eval_metric='mlogloss',
                      use_label_encoder=False,
                      num_class=3)

model.fit(X_train, y_train)
preds = model.predict(X_test)
print(confusion_matrix(y_test, preds))

[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]


In [52]:
# récupération des poids
dtf = model.get_booster().trees_to_dataframe()
dtf = dtf[dtf.Feature == 'Leaf']

# récupération des poids proches de zéro
dtf = dtf.sort_values(by=['Tree', 'Node', 'Gain'])
print('Noeuds égaux à poids 0 :', dtf[abs(dtf.Gain) == 0.0].shape[0])

Noeuds égaux à poids 0 : 0


In [53]:
# Régularisation de type L1 - modèle creux : alpha = 2
model1 = XGBClassifier(n_estimators=20,
                      learning_rate=0.3,
                      max_depth=3,
                      gamma=0,
                      reg_alpha=2,
                      reg_lambda=0,
                      eval_metric='mlogloss',
                      use_label_encoder=False,
                      num_class=3)

model1.fit(X_train, y_train)
preds1 = model1.predict(X_test)
print(confusion_matrix(y_test, preds))

[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]


In [58]:
dtf1 = dtf

# récupération des poids
dtf = model1.get_booster().trees_to_dataframe()
dtf = dtf[dtf.Feature == 'Leaf']
# récupération des poids proches de zéro
dtf = dtf.sort_values(by=['Tree', 'Node', 'Gain'])
print('Noeuds égaux à poids 0 :', dtf[abs(dtf.Gain) == 0.0].shape[0])

Noeuds égaux à poids 0 : 65


Il y a maintenant 65 poids qui ont une valeur nulle.

Ainsi, le premier modèle utilise moins de poids nuls et utilise donc des valeurs très proches de 0.

Tandis qu'en régularisant, avec un alpha positif, le modèle est beaucoup plus creux (contiennent bien des 0).

On favorisera un modèle creux plutôt que dense car les 0 ne seront pas stockés, c'est donc un gain de stockage.

Le fait de garder des poids différents de 0, les régularisations de type L1 pour alors sélectionner les caractéristiques les plus discriminantes.